In [2]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score, learning_curve, ShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

In [5]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, 
                        train_sizes=np.linspace(.1, 1.0, 5), figsize=(10,8)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure(figsize=figsize)
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [6]:
def plot_roc_curve(title, labels, predictions, figsize=(10, 8)):
    
    fpr, tpr, thrsh = roc_curve(labels, predictions)
    roc_auc = roc_auc_score(labels, predictions)
    
    plt.figure(figsize=figsize)
    plt.grid()
    plt.plot(fpr, tpr, color='b', label='ROC curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('{} Receiver Operating Characteristic'.format(title))
    plt.legend()

# Prep Data

In [7]:
df = pd.read_csv('bank/bank-additional-full.csv', sep=';')

In [35]:
df.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,1,0,0,0,1,0,0,0,1,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,1,0,0,0,1,0,0,0,1,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,1,0,0,0,1,0,0,0,1,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,1,0,0,0,1,0,0,0,1,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,1,0,0,0,1,0,0,0,1,0


In [31]:
categorical = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

In [34]:
df = pd.get_dummies(df, columns=categorical, drop_first=True)

In [36]:
x = df.drop('y', axis=1)
y = df[['y']]

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True, random_state=24)

# Decision Tree

In [39]:
dt = DecisionTreeClassifier(random_state=24)

In [40]:
dt.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=24,
            splitter='best')

In [41]:
dt.tree_.max_depth

33

In [42]:
dt_params = {'criterion': ['gini', 'entropy'],
             'splitter': ['best', 'random'],
             'max_depth': [15, 20, 25],
             'min_samples_split': [5, 10, 25], 
             'min_samples_leaf': [2, 5, 10],
             'max_features': ['auto', 'log2', None]}

In [43]:
dt_gs = GridSearchCV(dt, dt_params, cv=3, verbose=True, return_train_score=True, scoring='accuracy', n_jobs=-1)

In [44]:
dt_gs.fit(x_train, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   38.8s
[Parallel(n_jobs=-1)]: Done 972 out of 972 | elapsed:   45.2s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=24,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random'], 'max_depth': [15, 20, 25], 'min_samples_split': [5, 10, 25], 'min_samples_leaf': [2, 5, 10], 'max_features': ['auto', 'log2', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=True)

In [ ]:
dt_title = 'Decision Tree Learning Curves'
dt_cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=24)
plot_learning_curve(dt_gs, dt_title, x, y, cv=dt_cv, n_jobs=4, figsize=(10,8))

In [ ]:
plot_roc_curve('Decision Tree', y_test, dt_gs.predict(x_test))

In [ ]:
confusion_matrix(y_test, dt_gs.predict(x_test))

# SVM

In [ ]:
svm = SVC(random_state=24)

In [ ]:
svm_params = {'C': [1, 0.9, 0.8],
              'kernel': ['linear', 'poly', 'rbf']}

In [ ]:
svm_gs = GridSearchCV(svm, svm_params, cv=3, verbose=True, return_train_score=True, scoring='accuracy', n_jobs=4)

In [ ]:
svm_gs.fit(x_train, y_train.values.reshape(-1))

In [ ]:
svm_title = 'SVM Learning Curves'
svm_cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=24)
plot_learning_curve(svm_gs, svm_title, x, y, cv=svm_cv, n_jobs=4, figsize=(10,8))

In [ ]:
plot_roc_curve(y_test, svm_gs.predict(x_test))

In [ ]:
confusion_matrix(y_test, svm_gs.predict(x_test))

# KNN

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn_params = {'n_neighbors': [3, 5, 7],
              'weights': ['uniform', 'distance'],
              'algorithm': ['ball_tree', 'kd_tree', 'brute'],
              'p': [1, 2, 3]}

In [ ]:
knn_gs = GridSearchCV(knn, knn_params, cv=3, verbose=True, return_train_score=True, scoring='accuracy', n_jobs=4)

In [ ]:
knn_gs.fit(x_train, y_train)

In [ ]:
knn_title = 'KNN Learning Curves'
knn_cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=24)
plot_learning_curve(knn_gs, knn_title, x, y, cv=knn_cv, n_jobs=4, figsize=(10,8))

In [ ]:
plot_roc_curve(y_test, knn_gs.predict(x_test))

In [ ]:
confusion_matrix(y_test, knn_gs.predict(x_test))

# ANN

In [ ]:
ann = MLPClassifier()

In [ ]:
ann_params = {'hidden_layer_sizes': list(itertools.combinations_with_replacement([2**i for i in range(3, 8)], 2)),
              'activation': ['logistic', 'tanh', 'relu'],
              'learning_rate': ['constant', 'invscaling', 'adaptive']}

In [ ]:
scaler = StandardScaler()

In [ ]:
ann_gs = GridSearchCV(ann, ann_params, cv=3, verbose=True, return_train_score=True, scoring='accuracy', n_jobs=4)

In [ ]:
ann_gs.fit(scaler.fit_transform(x_train), y_train.values.reshape(-1))

In [ ]:
ann_title = 'ANN Learning Curves'
ann_cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=24)
plot_learning_curve(ann_gs.best_estimator_, ann_title, x, y, cv=ann_cv, n_jobs=4, figsize=(10,8))

In [ ]:
plot_roc_curve(y_test, ann_gs.predict(scaler.transform(x_test)))

In [ ]:
confusion_matrix(y_test, ann_gs.predict(x_test))

# Boosting

In [ ]:
bst = GradientBoostingClassifier()

In [ ]:
bst_params = {'loss': ['deviance', 'exponential'],
              'n_estimators': [10, 50, 100, 500],
              'subsample': [0.5, 0.75, 1.],
              'min_samples_split': [0.8, 0.9, 1.0]}

In [ ]:
bst_gs = GridSearchCV(bst, bst_params, cv=3, verbose=True, return_train_score=True, scoring='accuracy', n_jobs=4)

In [ ]:
bst_gs.fit(x_train, y_train.values.reshape(-1))

In [ ]:
bst_title = 'Boosting Learning Curves'
bst_cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=24)
plot_learning_curve(bst_gs, bst_title, x, y, cv=bst_cv, n_jobs=4, figsize=(10,8))

In [ ]:
plot_roc_curve(y_test, bst_gs.best_estimator_.decision_function(x_test))

In [ ]:
confusion_matrix(y_test, bst_gs.predict(x_test))